In [36]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from tensorflow.keras.datasets import imdb
from nltk.tokenize import word_tokenize

import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import numpy as np

import sqlite3

In [32]:
ckey = "DnAPBR25Wo9g2mAbbQe7jNnJf"
csecret = "XBSR3bwR9Eadk1ykDb9ScKZKvm5p2CDjf69mYnsR8RuBTHIiYh"

In [33]:
atoken = "1179559654700941312-UUh5kKPHp8sA5e6YYFwl7NpBNzz7C3"
asecret = "4JeWZ2lIMcH0rvbD5eWsWmwVnmgcdh4b8phaUcQ3LAC9K"

In [38]:
conn = sqlite3.connect('twitter_AAPL.db')

In [40]:
c = conn.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, sentiment REAL)")
    conn.commit()

create_table()

In [41]:
word_to_index = imdb.get_word_index()
loaded_model = load_model('GRU_all_vocab_model.h5')

def sentiment(tweet):
    word_token = word_tokenize(tweet)
    target_text = []
    for word in word_token:
        # 단어를 소문자로 변환 => 만약에 영화리뷰 감정 분석에 쓰였던 단어면 포함
        # 아니면 제외
        if word.lower() in word_to_index:
            # 단어를 word_to_index에서 찾아서, 숫자로 변환 
#             if word_to_index[word.lower()] < 10000:
                target_text.append(word_to_index[word.lower()])
    if len(target_text) == 0:
        return None # 분석할 단어가 없어서 감정 분석 불가능
    
    print(len(target_text))
    x_text = np.array([target_text])
    
    max_len = 500
    X_train = pad_sequences(x_text, maxlen = max_len)
    
    try:
        predicted = loaded_model.predict(X_train)
    except:
        return None

    return predicted

In [47]:
c.execute("DELETE FROM sentiment")

In [48]:
conn.commit()

In [ ]:
class listener(StreamListener):
    def on_data(self, data):
        # twitter에서 업데이트 된 twit이 data로 저장.
        # 업데이트 된 twit의 text가 긍정인지, 부정인지를 판단
        # 각 twit이 긍정이었는지, 부정이었는지 저장. 
        # 위 데이터가 시간대별로 쌓이면 => 어떤 주식 sentiment가 어떻게 변했는지 그리고 어떻게 변화하고 있는지.
        # json
        all_data = json.loads(data)
        
        tweet = all_data["text"]
        sentiment_value = sentiment(tweet)
        print(sentiment_value)
        if sentiment_value != None:
            time_ms = all_data['timestamp_ms']
            c.execute("INSERT INTO sentiment (unix, tweet, sentiment) VALUES (?,?,?)", (time_ms, tweet, float(sentiment_value[0][0])))
            conn.commit()
            output = open("twitter-AAPL.txt", "a")
            output.write(str(sentiment_value[0][0]))
            output.write('\n')
            output.close()
        return True
    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitter_stream = Stream(auth, listener())
twitter_stream.filter(track=["AAPL"])

3
[[0.39061442]]
8
[[2.6056246e-05]]
14
[[0.9867359]]
10
[[0.04674029]]
12
[[0.00398606]]
18
[[0.4907135]]
22
[[0.9521588]]
10
[[0.09600449]]
10
[[0.88255835]]
16
[[6.361341e-05]]
12
[[0.06565759]]
None
3
[[0.816676]]
11
[[0.00062999]]
16
[[0.00068885]]
17
[[0.00153407]]
11
[[0.99874145]]
7
[[0.48797554]]
4
[[0.00011518]]
5
[[7.074033e-05]]
21
[[0.99870634]]
18
[[0.19841924]]
20
[[0.00710285]]
22
[[0.00098839]]
19
[[0.8960464]]
19
[[0.99163294]]
22
[[4.0863792e-06]]
14
[[0.0001119]]
11
[[0.00603655]]
18
[[0.9891906]]
12
[[0.00398606]]
13
[[0.0011678]]
11
[[0.00062999]]
11
[[0.00062999]]
13
[[0.00205901]]
11
[[0.01258591]]
11
[[0.00062999]]
11
[[0.01258591]]
9
[[0.00370309]]
23
[[0.07347062]]
8
[[0.02590802]]
24
[[0.00426859]]
12
[[0.05278149]]
1
[[0.69687325]]
1
[[0.69687325]]
13
[[0.0011678]]
18
[[0.9584085]]
20
[[0.9109911]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
13
[[0.0011678]]
None
1
[[0.69687325]]
16
[[0.3242784]]
11
[[0.00062999]]
8
[[0.00416127]]
1
[[0.69687325]]
8
